# Maps of trend per grid cell

In [12]:
# import sys; sys.path.append('/rds/general/user/cb2714/home/wwa'); from wwa import *
import xarray as xr, numpy as np, pandas as pd, glob

import warnings; warnings.filterwarnings("ignore", category = FutureWarning) # otherwise there will be a LOT of warnings

import matplotlib, matplotlib.pyplot as plt

varnm = "tn14x"
ds_nm = "eobs"

def vec2map(x, mask):
    
    # reconstruct vector into map
    # create an empty map with NA in same cells as masks
    arr = mask.where(np.isnan(mask), 0)
    
    # get coordinates of non-empty cells
    px = np.argwhere(~np.isnan(mask.values))
    
    # Transfer vector values into non-empty cells in array
    if len(px) == len(x):
        for i in list(range(len(px))): arr[px[i,0], px[i,1]] = x[i]
        return arr
    else:
        print(str(len(x))+" values, but "+str(len(px))+" cells")
        return

## Reshape dataframe into maps

In [14]:
# load target map showing original shape of data
tmplt = xr.open_dataset("map-tmplt_"+varnm+"_"+ds_nm+".nc")
tmplt = tmplt[list(tmplt.data_vars)[0]]

# load all the flattened results and compile into a single dataframe
df = pd.concat([pd.read_csv(fnm, index_col = 0) for fnm in sorted(glob.glob("flattened-res/"+varnm+"*"+ds_nm+"*"))], axis = 1)

# reconstruct map
res = xr.merge([vec2map(df.loc[r], tmplt).rename(r) for r in df.index])

# replace infinite values with very high value
res["PR"] = res.PR.where(np.isfinite(res.PR), 100).where(~np.isnan(res.PR))

# save
res.to_netcdf("gridded-trend_"+varnm+"_"+ds_nm+".nc")